In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"beerukuriabhinay","key":"3c6f0c9ad9ec32be0b2ee0a90fec8fc9"}'}

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [4]:
!kaggle datasets download -d beerukuriabhinay/nlp-asr

100% 1.85G/1.85G [00:20<00:00, 104MB/s]
100% 1.85G/1.85G [00:20<00:00, 95.4MB/s]


In [1]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install evaluate
!pip install accelerate -U

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-6j99fhhj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-6j99fhhj
  Resolved https://github.com/huggingface/datasets.git to commit 29ffc270da34de70cf8e28b2ebeadba1c06d8730
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 10.7 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-2.18.1.dev0-py3-none-any.whl size=513976 sha256=6107f233b18ab439c9b5d66fa33c1a73e3c822312721f644fab7a23aca09954d
  Stored in directory: /tmp/pip-ephem-wheel-cache-i0uxhov

In [5]:
import zipfile
zip_ref = zipfile.ZipFile('nlp-asr.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
!ls

Athervavedha  nlp-asr.zip  sample_data


In [25]:
import pandas as pd

# Path to your text file
file_path = 'Athervavedha/Mapping/output.txt'  # Replace '/path/to/your/file.txt' with the actual file path


data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('       ', 1)
        if len(parts) == 2:
            data.append(parts)

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=['path', 'text'])

In [26]:
len(df)

9997

In [27]:
df.head()

,path,text
0,Atharvaveda_Kanda_1_0001.wav,ये त्रिषप्ताः परियन्ति विश्वा रूपाणि बिभ्रतः
1,Atharvaveda_Kanda_1_0002.wav,वाचस्पतिर्बला तेषां तन्वो अद्य दधातु मे
2,Atharvaveda_Kanda_1_0003.wav,पुनरेहि वचस्पते देवेन मनसा सह
3,Atharvaveda_Kanda_1_0004.wav,वसोष्पते नि रमय मय्येवास्तु मयि श्रुतम्
4,Atharvaveda_Kanda_1_0005.wav,इहैवाभि वि तनूभे आर्त्नी इव ज्यया


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
train_data = train_data[:6000]
test_data = test_data[ : 400]

In [31]:
# Function to remove specified characters from text
def remove_chars(text):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\[\5\1\|]'
    return re.sub(chars_to_ignore_regex, '', text)

In [32]:
# Applying the function to the 'text' column
train_data['text'] = train_data['text'].apply(remove_chars)
# Applying the function to the 'text' column
test_data['text'] = test_data['text'].apply(remove_chars)

In [33]:
print(len(train_data))

6000


In [34]:
print(len(test_data))

400


In [35]:
train_data.head()

,path,text
6715,Atharvaveda_Part_015_0131.wav,ऐनं प्रियं गच्छति प्रियः प्रियस्य भवति य एवं वेद
1561,Atharvaveda_Kanda_5_0101.wav,पर्यू षु प्र धन्वा वाजसातये परि वृत्राणि सक्षणिः
1670,Atharvaveda_Kanda_5_0210.wav,आ सुष्वयन्ती यजते उपाके उषासानक्ता सदतां नि योनौ
4345,Atharvaveda_Kanda_10_0044.wav,दुष्कृते विद्युतं देवहेतिम्
8098,Atharvaveda_Part_019_2_0351.wav,सायंसायं गृहपतिर्नो अग्निः प्रातःप्रातः सौमनसस...


In [36]:
test_data = test_data.copy()
train_data = train_data.copy()

In [37]:
# add file path
def add_file_path(text):
    text = "Athervavedha/Audio_files/" + text
    return text

In [38]:
test_data.head()

,path,text
4122,Atharvaveda_Kanda_9_0264.wav,यजमानब्राह्मणं वा एतदतिथिपतिः कुरुते यदाहार्या...
4065,Atharvaveda_Kanda_9_0207.wav,येना सहस्रं वहसि येनाग्ने सर्ववेदसम्
1731,Atharvaveda_Kanda_5_0271.wav,षट्च मे षष्टिश्च मेऽपवक्तार ओषधे ऋतजात ऋतावरि...
4740,Atharvaveda_Kanda_10_0439.wav,तिस्रो ह प्रजा अत्यायमायन् न्यन्या अर्कमभितोऽव...
6391,Atharvaveda_Part_014_0261.wav,एवा त्वं सम्राज्ञ्येधि पत्युरस्तं परेत्य


In [39]:
# add file path
train_data.loc[: , 'audio_path'] = train_data['path'].map(lambda x: add_file_path(x))
test_data.loc[: , 'audio_path'] = test_data['path'].map(lambda x: add_file_path(x))

In [40]:
test_data.reset_index( drop = True , inplace = True)

In [41]:
test_data.drop(columns = ["path"] , inplace = True)
train_data.drop(columns = ["path"], inplace = True)

In [42]:
test_data.iloc[0,:]

text          यजमानब्राह्मणं वा एतदतिथिपतिः कुरुते यदाहार्या...
audio_path    Athervavedha/Audio_files/Atharvaveda_Kanda_9_0...
Name: 0, dtype: object

In [43]:
batch = test_data["text"][0]

In [44]:
from datasets import Dataset

In [45]:
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

In [20]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\[\5\1\|]'  # TODO: adapt this list to include all special characters you removed from the data

In [46]:
import re
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [47]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

{'व': 0, 'इ': 1, '5': 2, 'घ': 3, 'म': 4, 'ऐ': 5, 'औ': 6, 'ौ': 7, 'द': 8, 'ॄ': 9, '१': 10, 'ओ': 11, 'ि': 12, 'ँ': 13, 'भ': 14, 'ठ': 15, 'ञ': 16, 'ट': 17, 'य': 18, 'र': 19, 'छ': 20, '्': 21, 'ळ': 22, 'ष': 23, 'ल': 24, 'प': 25, 'उ': 26, 'ै': 27, '३': 28, 'ी': 29, 'ः': 30, 'च': 31, 'े': 32, 'स': 33, 'ध': 34, 'ृ': 35, 'ज': 36, '0': 37, 'ऽ': 38, 'ू': 39, 'ख': 40, '1': 41, 'ु': 42, ' ': 43, 'ब': 44, 'त': 45, 'न': 46, 'ो': 47, 'ं': 48, 'ड': 49, 'ग': 50, 'फ': 51, '५': 52, 'ण': 53, 'ए': 54, 'ङ': 55, 'क': 56, 'आ': 57, 'ऊ': 58, 'ढ': 59, 'ई': 60, 'अ': 61, 'थ': 62, 'ह': 63, 'ॢ': 64, 'ऋ': 65, 'श': 66, 'ा': 67}


In [48]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [49]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

70


In [50]:
print(vocab_dict)

{'व': 0, 'इ': 1, '5': 2, 'घ': 3, 'म': 4, 'ऐ': 5, 'औ': 6, 'ौ': 7, 'द': 8, 'ॄ': 9, '१': 10, 'ओ': 11, 'ि': 12, 'ँ': 13, 'भ': 14, 'ठ': 15, 'ञ': 16, 'ट': 17, 'य': 18, 'र': 19, 'छ': 20, '्': 21, 'ळ': 22, 'ष': 23, 'ल': 24, 'प': 25, 'उ': 26, 'ै': 27, '३': 28, 'ी': 29, 'ः': 30, 'च': 31, 'े': 32, 'स': 33, 'ध': 34, 'ृ': 35, 'ज': 36, '0': 37, 'ऽ': 38, 'ू': 39, 'ख': 40, '1': 41, 'ु': 42, 'ब': 44, 'त': 45, 'न': 46, 'ो': 47, 'ं': 48, 'ड': 49, 'ग': 50, 'फ': 51, '५': 52, 'ण': 53, 'ए': 54, 'ङ': 55, 'क': 56, 'आ': 57, 'ऊ': 58, 'ढ': 59, 'ई': 60, 'अ': 61, 'थ': 62, 'ह': 63, 'ॢ': 64, 'ऋ': 65, 'श': 66, 'ा': 67, '|': 43, '[UNK]': 68, '[PAD]': 69}


In [51]:
print(len(vocab_dict))

70


In [52]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [53]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [54]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [55]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [56]:
processor.save_pretrained("wav2vec2-large-xlsr-hindi-demo/")

[]

In [57]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\[\5\1\|]'  # TODO: adapt this list to include all special characters you removed from the data

In [58]:
import librosa
import re
def speech_file_to_array_fn(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"])
    speech_array, sampling_rate = librosa.load(batch["audio_path"], sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [59]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [60]:
test_data = test_data.map(speech_file_to_array_fn, remove_columns=test_data.column_names)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [61]:
data = next(iter(train_data))

In [62]:
data["sampling_rate"]

16000

In [63]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [64]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names, batch_size=4, batched=True)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names, batch_size=4,  batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [65]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        return batch

In [66]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [67]:
from evaluate import load

In [68]:
wer_metric = load("wer")

In [69]:
import numpy as np

In [108]:
processor.tokenizer.pad_token_id

69

In [70]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [71]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # This will print whether a GPU is being used.


Using device: cuda


In [72]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook

In [73]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1920: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [74]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-large-xlsr-hindi-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=5,
  fp16=torch.cuda.is_available(),
  save_steps=700,
  eval_steps=700,
  logging_steps=700,
  learning_rate=2e-3,
  warmup_steps=500,
  save_total_limit=1,
)

In [75]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [76]:
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }
# setInterval(ConnectButton,60000);

In [77]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
700,4.167900,2.504278,0.999630
1400,1.939800,1.465849,0.996300
2100,1.478000,1.208320,0.991861
2800,1.275800,1.076738,0.980022
3500,1.133500,0.995028,0.960044


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=3750, training_loss=1.9375422281901042, metrics={'train_runtime': 5672.9151, 'train_samples_per_second': 5.288, 'train_steps_per_second': 0.661, 'total_flos': 5.910391428352969e+18, 'train_loss': 1.9375422281901042, 'epoch': 5.0})

In [78]:
model = Wav2Vec2ForCTC.from_pretrained("/content/wav2vec2-large-xlsr-hindi-demo/checkpoint-3500").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/wav2vec2-large-xlsr-hindi-demo")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [79]:
wer = load("wer")

In [114]:
input_dict = processor(test_data["input_values"][2], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [103]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(processor.decode(test_data["labels"][0]))

Prediction:
यचमानब्रां्भणं वयेपतदितिपरिः कुरुतेयातहार्यनिप्रेक्षत रिदं भूया

Reference:
यजमानब्राह्मणं वा एतदतिथिपतिः कुरुते यदाहार्याणि प्रेक्षत इदं भूया३


In [105]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(processor.decode(test_data["labels"][1]))

Prediction:
ेसहस्नरं महसि येनणाङ् यनेसर्ववेचसम्

Reference:
येना सहस्रं वहसि येनाग्ने सर्वेदसम्
